In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [59]:
#df_P = pd.read_csv(filepath_or_buffer = 'D3_pre.csv', parse_dates = [[1,2]], index_col = False)  
#df_P = df_P.set_index('YYYY/MM/DD_HH:MM:SS')
#df_P.drop(df_P.columns[[0,2,3,4,5,6,9,10,11,12,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40]], axis=1, inplace=True)
df_P = pd.io.parsers.read_csv(filepath_or_buffer = 'data/D0_filt_10mino3.csv',index_col = 0)
print df_P[:5]

                       O3_ppb    UnixTime       e2v03       Temp         Rh  \
DATE (MM/DD/YYYY)_MST                                                         
7/10/14 20:25              41  1405023930  114.714286  45.800000  11.400000   
7/10/14 20:26              39  1405023991  120.428571  45.814286  11.314286   
7/10/14 20:27              44  1405024053  119.142857  45.857143  11.300000   
7/10/14 20:28              47  1405024110  117.500000  45.900000  11.300000   
7/10/14 20:29              44  1405024167  120.285714  45.900000  11.457143   

                       Zenith Angle [degrees]  days from start  ref_o3_smooth  \
DATE (MM/DD/YYYY)_MST                                                           
7/10/14 20:25                        99.48960                0             41   
7/10/14 20:26                        99.63789                0             39   
7/10/14 20:27                        99.78580                0             44   
7/10/14 20:28                        99.9

In [60]:
df_P.drop(df_P.columns[[7,8]], axis=1, inplace=True)
print df_P[:5]

                       O3_ppb    UnixTime       e2v03       Temp         Rh  \
DATE (MM/DD/YYYY)_MST                                                         
7/10/14 20:25              41  1405023930  114.714286  45.800000  11.400000   
7/10/14 20:26              39  1405023991  120.428571  45.814286  11.314286   
7/10/14 20:27              44  1405024053  119.142857  45.857143  11.300000   
7/10/14 20:28              47  1405024110  117.500000  45.900000  11.300000   
7/10/14 20:29              44  1405024167  120.285714  45.900000  11.457143   

                       Zenith Angle [degrees]  days from start  
DATE (MM/DD/YYYY)_MST                                           
7/10/14 20:25                        99.48960                0  
7/10/14 20:26                        99.63789                0  
7/10/14 20:27                        99.78580                0  
7/10/14 20:28                        99.93332                0  
7/10/14 20:29                       100.08045           

####Define the slope function.

In [61]:
def find_mean_slope_over_time(df_P, int_min, data_col):
    df_P['diff'] = df_P[data_col].diff()
    
    slope = []
    for i in range(int_min,len(df_P['diff'])):
        top = i-int_min
        slope.append(df_P['diff'][top:i].mean())
    
    for i in range(0,int_min):
        slope.insert(i, 'NaN')

    return slope

####Define functions that create features that represent the area under the temp and humidity curves.

In [62]:
def make_lag_integral_features(df_P, int_min, data_col):
    area_curve = []

    for i in range(int_min,len(df_P[data_col])):
        top = i - int_min
        area_curve.append(np.trapz(df_P[data_col][top:i]))
    
    for i in range(0,int_min):
        area_curve.insert(i, 'NaN')
    
    return area_curve

In [63]:
def make_lead_integral_features(df_P, int_min, data_col):
    area_curve = []

    for i in range(0,len(df_P[data_col])-int_min):
        top = i + int_min
        area_curve.append(np.trapz(df_P[data_col][i:top]))
    
    for i in range(len(df_P[data_col])-int_min, len(df_P[data_col])):
        area_curve.insert(i, 'NaN')
    
    return area_curve

In [64]:
df_P['o3_lead_30'] = make_lead_integral_features(df_P, 30,'e2v03')

###Call the slope functions and add them to the dataframe.

In [65]:
df_P['o3_slope_105'] = find_mean_slope_over_time(df_P, 105,'e2v03')
df_P['o3_slope_98'] = find_mean_slope_over_time(df_P, 98,'e2v03')
df_P['o3_slope_90'] = find_mean_slope_over_time(df_P, 90,'e2v03')
df_P['o3_slope_83'] = find_mean_slope_over_time(df_P, 83,'e2v03')
df_P['o3_slope_75'] = find_mean_slope_over_time(df_P, 75,'e2v03')
df_P['o3_slope_68'] = find_mean_slope_over_time(df_P, 60,'e2v03')
df_P['o3_slope_60'] = find_mean_slope_over_time(df_P, 60,'e2v03')
df_P['o3_slope_53'] = find_mean_slope_over_time(df_P, 53,'e2v03')
df_P['o3_slope_45'] = find_mean_slope_over_time(df_P, 45,'e2v03')
df_P['o3_slope_38'] = find_mean_slope_over_time(df_P, 38,'e2v03')
df_P['o3_slope_30'] = find_mean_slope_over_time(df_P, 30,'e2v03')
df_P['o3_slope_23'] = find_mean_slope_over_time(df_P, 23,'e2v03')
df_P['o3_slope_15'] = find_mean_slope_over_time(df_P, 15,'e2v03')
df_P['o3_slope_10'] = find_mean_slope_over_time(df_P, 10,'e2v03')
df_P['o3_slope_5'] = find_mean_slope_over_time(df_P, 5,'e2v03')

df_P['temp_slope_90'] = find_mean_slope_over_time(df_P, 90,'Temp')
df_P['temp_slope_75'] = find_mean_slope_over_time(df_P, 75,'Temp')
df_P['temp_slope_60'] = find_mean_slope_over_time(df_P, 60,'Temp')
df_P['temp_slope_45'] = find_mean_slope_over_time(df_P, 45,'Temp')
df_P['temp_slope_30'] = find_mean_slope_over_time(df_P, 30,'Temp')
df_P['temp_slope_15'] = find_mean_slope_over_time(df_P, 15,'Temp')
df_P['temp_slope_5'] = find_mean_slope_over_time(df_P, 5,'Temp')

df_P['rh_slope_90'] = find_mean_slope_over_time(df_P, 90,'Rh')
df_P['rh_slope_75'] = find_mean_slope_over_time(df_P, 75,'Rh')
df_P['rh_slope_60'] = find_mean_slope_over_time(df_P, 60,'Rh')
df_P['rh_slope_45'] = find_mean_slope_over_time(df_P, 45,'Rh')
df_P['rh_slope_30'] = find_mean_slope_over_time(df_P, 30,'Rh')
df_P['rh_slope_15'] = find_mean_slope_over_time(df_P, 15,'Rh')
df_P['rh_slope_5'] = find_mean_slope_over_time(df_P, 5,'Rh')

###Call the integration functions and add them to the dataframe.

In [66]:
#df_P['rh_int_105'] = make_time_integral_features(df_P,105,'Rh')
df_P['rh_lag_int_90'] = make_lag_integral_features(df_P,90,'Rh')
df_P['rh_lag_int_75'] = make_lag_integral_features(df_P,75,'Rh')
df_P['rh_lag_int_60'] = make_lag_integral_features(df_P,60,'Rh')
df_P['rh_lag_int_45'] = make_lag_integral_features(df_P,45,'Rh')
df_P['rh_lag_int_30'] = make_lag_integral_features(df_P,30,'Rh')

#df_P['temp_int_105'] = make_time_integral_features(df_P,105,'Temp')
df_P['temp_lag_int_90'] = make_lag_integral_features(df_P,90,'Temp')
df_P['temp_lag_int_75'] = make_lag_integral_features(df_P,75,'Temp')
df_P['temp_lag_int_60'] = make_lag_integral_features(df_P,60,'Temp')
df_P['temp_lag_int_45'] = make_lag_integral_features(df_P,45,'Temp')
df_P['temp_lag_int_30'] = make_lag_integral_features(df_P,30,'Temp')

df_P['o3_lag_int_90'] = make_lag_integral_features(df_P,90,'e2v03')
df_P['o3_lag_int_75'] = make_lag_integral_features(df_P,75,'e2v03')
df_P['o3_lag_int_60'] = make_lag_integral_features(df_P,60,'e2v03')
df_P['o3_lag_int_45'] = make_lag_integral_features(df_P,45,'e2v03')
df_P['o3_lag_int_30'] = make_lag_integral_features(df_P,30,'e2v03')

#df_P['rh_int_105'] = make_time_integral_features(df_P,105,'Rh')
df_P['rh_lead_int_90'] = make_lead_integral_features(df_P,90,'Rh')
df_P['rh_lead_int_75'] = make_lead_integral_features(df_P,75,'Rh')
df_P['rh_lead_int_60'] = make_lead_integral_features(df_P,60,'Rh')
df_P['rh_lead_int_45'] = make_lead_integral_features(df_P,45,'Rh')
df_P['rh_lead_int_30'] = make_lead_integral_features(df_P,30,'Rh')
df_P['rh_lead_int_15'] = make_lead_integral_features(df_P,15,'Rh')
df_P['rh_lead_int_10'] = make_lead_integral_features(df_P,10,'Rh')
df_P['rh_lead_int_5'] = make_lead_integral_features(df_P,5,'Rh')

#df_P['temp_int_105'] = make_time_integral_features(df_P,105,'Temp')
df_P['temp_lead_int_90'] = make_lead_integral_features(df_P,90,'Temp')
df_P['temp_lead_int_75'] = make_lead_integral_features(df_P,75,'Temp')
df_P['temp_lead_int_60'] = make_lead_integral_features(df_P,60,'Temp')
df_P['temp_lead_int_45'] = make_lead_integral_features(df_P,45,'Temp')
df_P['temp_lead_int_30'] = make_lead_integral_features(df_P,30,'Temp')
df_P['temp_lead_int_15'] = make_lead_integral_features(df_P,15,'Temp')
df_P['temp_lead_int_10'] = make_lead_integral_features(df_P,10,'Temp')
df_P['temp_lead_int_5'] = make_lead_integral_features(df_P,5,'Temp')

df_P['o3_lead_int_90'] = make_lead_integral_features(df_P,90,'e2v03')
df_P['o3_lead_int_75'] = make_lead_integral_features(df_P,75,'e2v03')
df_P['o3_lead_int_60'] = make_lead_integral_features(df_P,60,'e2v03')
df_P['o3_lead_int_45'] = make_lead_integral_features(df_P,45,'e2v03')
df_P['o3_lead_int_30'] = make_lead_integral_features(df_P,30,'e2v03')
df_P['o3_lead_int_15'] = make_lead_integral_features(df_P,15,'e2v03')
df_P['o3_lead_int_10'] = make_lead_integral_features(df_P,10,'e2v03')
df_P['o3_lead_int_5'] = make_lead_integral_features(df_P,5,'e2v03')

In [67]:
#Delete the first 90 rows of the dataframe to remove all NaNs.
df_P = df_P.ix[105:len(df_P['e2v03'])-90]
print df_P.ix[len(df_P['e2v03'])-90:len(df_P['e2v03'])]

                       O3_ppb    UnixTime      e2v03       Temp        Rh  \
DATE (MM/DD/YYYY)_MST                                                       
9/3/14 21:00               39  1409778030  93.000000  46.800000  8.500000   
9/3/14 21:01               38  1409778092  93.142857  46.800000  8.528571   
9/3/14 21:02               39  1409778149  95.000000  46.800000  8.683333   
9/3/14 21:03               42  1409778207  95.857143  46.728571  8.628571   
9/3/14 21:04               44  1409778268  96.000000  46.800000  8.628571   
9/3/14 21:05               43  1409778330  95.428571  46.800000  8.514286   
9/3/14 21:06               45  1409778392  93.142857  46.800000  8.400000   
9/3/14 21:07               43  1409778449  93.000000  46.850000  8.400000   
9/3/14 21:08               44  1409778507  93.000000  46.814286  8.442857   
9/3/14 21:09               48  1409778568  93.571429  46.900000  8.428571   
9/3/14 21:10               46  1409778630  94.571429  46.900000  8.314286   

In [69]:
print df_P.ix[len(df_P['e2v03'])-5:len(df_P['e2v03'])]

                       O3_ppb    UnixTime      e2v03       Temp        Rh  \
DATE (MM/DD/YYYY)_MST                                                       
9/3/14 22:25               43  1409783131  82.714286  43.342857  9.085714   
9/3/14 22:26               26  1409783189  84.000000  43.283333  9.233333   
9/3/14 22:27               36  1409783246  84.571429  43.200000  9.371429   
9/3/14 22:28               41  1409783308  88.000000  43.114286  9.457143   
9/3/14 22:29               40  1409783370  90.142857  43.057143  9.442857   

                       Zenith Angle [degrees]  days from start o3_lead_30  \
DATE (MM/DD/YYYY)_MST                                                       
9/3/14 22:25                        128.19138               55   2562.155   
9/3/14 22:26                        128.28881               55   2565.429   
9/3/14 22:27                        128.38539               55   2565.786   
9/3/14 22:28                        128.48113               55   2565.214  

In [55]:
#df_P.drop(df_P.columns[[9]], axis=1, inplace=True)
#print df_P[:5]

In [70]:
#ozone terms
df_P['O3_sqrt'] = np.sqrt(df_P['e2v03'].astype(float))
df_P['O3_sq'] = df_P['e2v03']**2
df_P['O3_cu'] = df_P['e2v03']**3
df_P['ln_O3'] = np.log(df_P['e2v03'])


#temp terms
df_P['temp_sqrt'] = np.sqrt(df_P['Temp'].astype(float))
df_P['temp_sq'] = df_P['Temp']**2
df_P['temp_cu'] = df_P['Temp']**3
df_P['ln_temp'] = np.log(df_P['Temp'])


#rh terms
df_P['rh_sqrt'] = np.sqrt(df_P['Rh'].astype(float))
df_P['rh_sq'] = df_P['Rh']**2
df_P['rh_cu'] = df_P['Rh']**3
df_P['ln_rh'] = np.log(df_P['Rh'])


#ozone/temp interactions
df_P['03_mult_temp'] = df_P['e2v03']*df_P['Temp']
df_P['03_mult_temp_sq'] = (df_P['e2v03']*df_P['Temp'])**2
df_P['03_mult_temp_cu'] = (df_P['e2v03']*df_P['Temp'])**3
df_P['ln_03_mult_temp'] = np.log(df_P['03_mult_temp'])


#ozone/rh interactions
df_P['03_mult_rh'] = df_P['e2v03']*df_P['Rh']
df_P['03_mult_rh_sq'] = (df_P['e2v03']*df_P['Rh'])**2
df_P['03_mult_rh_cu'] = (df_P['e2v03']*df_P['Rh'])**3
df_P['ln_03_mult_rh'] = np.log(df_P['03_mult_rh'])

#temp/rh interactions
df_P['temp_mult_rh'] = df_P['Temp']*df_P['Rh']
df_P['temp_mult_rh_sq'] = (df_P['Temp']*df_P['Rh'])**2
df_P['temp_mult_rh_cu'] = (df_P['Temp']*df_P['Rh'])**3
df_P['ln_temp_mult_rh'] = np.log(df_P['temp_mult_rh'])


#ozone/rh/temp interactions
df_P['03_mult_rh_&_temp'] = df_P['e2v03']*df_P['Rh']*df_P['Temp']
df_P['03_mult_rh_&_temp_sq'] = (df_P['e2v03']*df_P['Rh']*df_P['Temp'])**2
df_P['03_mult_rh_&_temp_cu'] = (df_P['e2v03']*df_P['Rh']*df_P['Temp'])**3
df_P['ln_03_mult_rh_&_temp_cu'] = np.log(df_P['03_mult_rh_&_temp_cu'])

print df_P[:5]

                       O3_ppb    UnixTime       e2v03       Temp         Rh  \
DATE (MM/DD/YYYY)_MST                                                         
7/10/14 22:10              40  1405030229   97.666667  43.616667  11.650000   
7/10/14 22:11              38  1405030286  103.000000  43.671429  11.528571   
7/10/14 22:12              53  1405030348   97.142857  43.700000  11.442857   
7/10/14 22:13              54  1405030409   92.142857  43.757143  11.400000   
7/10/14 22:14              54  1405030470   90.857143  43.800000  11.400000   

                       Zenith Angle [degrees]  days from start o3_lead_30  \
DATE (MM/DD/YYYY)_MST                                                       
7/10/14 22:10                       112.46823                0   2694.333   
7/10/14 22:11                       112.56154                0   2687.643   
7/10/14 22:12                       112.65418                0   2677.357   
7/10/14 22:13                       112.74614                

In [72]:
df_P.to_csv(path_or_buf = 'data/D0_raw_features.csv')